In [ ]:
# 安装依赖
!pip install -U transformers tokenizers
!pip install vllm huggingface-hub flask pyngrok

# 挂载Google网盘
Mount_GDrive = True # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

# 运行API后端(vLLM)
# 使用ngrok进行API映射（ngrokToken留空则使用localtunnel）
from huggingface_hub import hf_hub_download

ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(8001)
      public_url = ssh_tunnel.public_url
      print('address：'+public_url)
  else:
      public_url = ssh_tunnels[0].public_url
      print('address：'+public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(8001,)).start()
  public_url = "http://localhost:8001"

%cd $ROOT_PATH

!RAY_memory_monitor_refresh_ms="0" HF_ENDPOINT=https://huggingface.co OMP_NUM_THREADS=36 VLLM_ATTENTION_BACKEND=XFORMERS vllm serve Backup_hf_sakura-14b-qwen2.5-v1.0-exl2 --tokenizer Qwen/Qwen2.5-14B-Instruct --dtype float16 --api-key token-abc123 --kv-cache-dtype auto --max-model-len 32768 --tensor-parallel-size 1 --gpu-memory-utilization 0.99 --disable-custom-all-reduce --enforce-eager --use-v2-block-manager --disable-log-requests --host 0.0.0.0 --port 8001 --served-model-name "Qwen2.5-14B-Instruct" &

print(f"API is accessible at {public_url}")